In [1]:
from idealab_tools.kinematics.kinematics import Quaternion
from typing import Dict, Union
import mujoco
from gymnasium import spaces
import gymnasium as gym
import gymnasium
import numpy as np
import numpy
from gymnasium import utils
from gymnasium.spaces import Box
import math
import os
import os
import mujoco
import numpy
import mediapy as media
import matplotlib.pyplot as plt
import math
import yaml

In [2]:
width = 800
height = 600

In [3]:
with open('my_envs/biped.xml') as f:
    xml_template = f.read()

In [4]:
xml = xml_template.format(l_ankle=.03,k=.1,b=.01,ts=1e-3,width=800,height=600)

In [5]:
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model,width=width,height=height)
# renderer = mujoco.Renderer(model)


libdecor-gtk-WARNING: Failed to initialize GTK
Failed to load plugin 'libdecor-gtk.so': failed to init
No plugins found, falling back on no decorations


In [6]:
A = .25
d = 0 
ts = .25
f = .5
l1 = 0
l2 = .5

def mycontroller(model, data):
    t = data.time
    linear_act = t*d/ts
    if linear_act < d:
        linear_act = d
    pos1 = A*math.sin(2*math.pi*(f*t-l1))
    pos2 = A*math.sin(2*math.pi*(f*t-l2))
    data.ctrl = [pos1,pos2]
    return


mujoco.set_mjcb_control(mycontroller)

In [7]:
mujoco.mj_resetData(model,data)
# data.qpos[0:2] = [-45*math.pi/180,-45*math.pi/180]
# data.qvel[0:2] = [-3,0]
frames=[]

duration = 5
framerate =30

q = []
w = []
t = []
xy = []

while data.time<duration:


    mujoco.mj_step(model,data)

    q.append(data.qpos.copy())
    w.append(data.qvel.copy())
    xy.append(data.xpos.copy())
    t.append(data.time)
    
    if len(frames)<data.time*framerate:
    
        # renderer.update_scene(data)
        renderer.update_scene(data,"world")
        pixels = renderer.render()
        frames.append(pixels)

mujoco.set_mjcb_control(None)
        
media.show_video(frames,fps = framerate,width=width,height=height)
# media.write_video('pendulum.mp4',frames,fps=framerate)

This browser does not support the video tag.


In [8]:
data.qpos

array([ 4.24114726e-02,  7.23997648e-04,  1.04686340e-01,  9.99987393e-01,
       -5.65589144e-05, -3.57372825e-03,  3.52685708e-03,  4.08088591e-02,
        7.72488357e-02, -1.09695817e-01, -4.09759305e-02, -5.99024400e-02,
        1.06570435e-01])

In [9]:
data.xpos

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.04241629,  0.00072321,  0.10467829],
       [ 0.0424241 ,  0.02072104,  0.08467612],
       [ 0.04104502,  0.0207066 ,  0.04469991],
       [ 0.0366063 ,  0.02067196,  0.00494696],
       [ 0.04268863, -0.01927808,  0.08468144],
       [ 0.04463763, -0.01927051,  0.04472896],
       [ 0.04896157, -0.0192472 ,  0.00496335],
       [ 0.        ,  0.        ,  0.        ]])